2. **Translation (with dictionary definitions as source):**
    - The translation feature allows the user to input a word or phrase in their language and get the translation in Greek.
    - Input: User's text in source language.
    - Output: Translated text in Greek.
    - The back-end uses `translation.py` script that utilizes the OpenAI API.
    - The front-end will have a form for users to input text and a display area to present translated text.
3. **Real-time Transcription and Translation:**
    - This feature transcribes spoken language and translates it into Greek in real-time.
    - Input: Spoken language through a microphone.
    - Output: Transcribed and translated text in Greek.
    - The back-end leverages `transcription.py` and `real_time_translation.py` scripts, interfacing with OpenAI API.
    - The front-end will require microphone access and display the transcribed and translated text.

Tranlsation requirements:
English to greek
 - identify context 
 - identify key words
 - translate text using key words and context to improve accuracy of the translation
 - include key words and translations beneath translation and save to vocab list with sentence as example for translation that can then be exported to a vocab list table

Greek to enlgish:
 - identify context 
 - identify key words
 - translate text using key words and context to improve accuracy of the translation
 - include key words and translations beneath translation and save to vocab list with sentence as example for translation that can then be exported to a vocab list table


Transcription and translate
  - Transcribe
  - Improve transcription quality with post-proofreading validation using grammar, syntax and in context semantic verification of coherence. 
  - Translate to english with correct in-context translation


Translate document
  - Map reduce and summarise document
  - chunk around paragraphs
  - batch send chunks and stream translation

In [2]:
import gradio as gr
import whisper
import os
from dotenv import load_dotenv
import openai
import logging


load_dotenv(".env")
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [3]:
# Create a logger
logging.basicConfig(filename='example.log',level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# Generic Chatgpt chat completition with input arguments for system message and user message

def chatbot_completition(system_message, user_message):
    response = openai.ChatCompletion.create(
        model = 'gpt-4',
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ],
    )
    
    prompt, reply = response['usage']['prompt_tokens'], response['usage']['completion_tokens']
    cost = (0.03/1000)*prompt + (0.06/1000)*reply
    ## calculate api costs and log it
    logger.info(f"Prompt tokens: {prompt}, Completion tokens: {reply}, Cost: {cost}")

    return response["choices"][0]["message"]["content"]

add conditionals on whether languge is en to gr or gr to en

ie a checkbox ' translate from grek to english'
and redeclare source and target language


start with default english to greek for source and target langauge

so translation needs a toggl for example output cases 

In [33]:
# Translation chain
class Translation:
    def __init__(self, source_language, target_language):
        self.source_language = source_language
        self.target_language = target_language
        
    def translate(self, text):
        logger.info("Starting translation...")
        
        logger.info("Extracting context...")
        context = self.extract_context(text)
        logger.info(f"Context extracted: {context}")
        
        logger.info("Extracting key words...")
        key_words = self.extract_key_words(text)
        logger.info(f"Key words extracted: {key_words}")
        
        SYSTEM = f"""You are an expert {self.target_language} translator. Your role is to carefully examine the given text, which may be written in {self.source_language}, and translate it accurately into {self.target_language}. Your translation work should account for not only literal translation but also cultural and contextual nuances. You'll be given text that has to be translated into {self.target_language}, the context of the text and key words with possible translations. Your task is to ensure the translated text preserves the original message while being grammatically correct and contextually appropriate in {self.target_language}. If the text is not in {self.source_language} you will say "Please enter text in {self.source_language} to translate" Respond using markdown."""
        
        example_en_to_gr = f"""                    
                    "Given this text to translate: "It's a beautiful sunny day at the beach."
                    Translate it from English to Greek:
                    Given the context: "This sentence is describing a pleasant weather condition at a seaside location." and key words: "beautiful (όμορφος), sunny (ηλιόλουστος), day (μέρα), beach (παραλία)"
                    The translation is: "Είναι μια όμορφη ηλιόλουστη μέρα στην παραλία." 
                    """
        example_gr_to_en = f"""
                    "Given this text to translate: "Είναι μια όμορφη ηλιόλουστη μέρα στην παραλία."
                    Translate it from Greek to English:
                    Given the context: "This sentence is describing a pleasant weather condition at a seaside location." and key words: "όμορφη (beautiful), ηλιόλουστη (sunny), μέρα (day), παραλία (beach)"
                    The translation is: "It's a beautiful sunny day at the beach." " 
                    """
                    
        if self.source_language == "English":
            example = example_en_to_gr
        else:
            example = example_gr_to_en
                
        PROMPT = f"""
                    Translate from {self.source_language} to {self.target_language}:
                    
                    Example:
                    {example}
                    
                    Now, given this text to translate: "{text}"
       
                    Given the context: "{context}" and key words: "{key_words}"
                    
                    The translation is:
                    """
        logger.info("Translating text...")
        translation = chatbot_completition(SYSTEM, PROMPT)
        logger.info(f"Translation completed: {translation}")
        
        return translation, key_words
        
    
    def extract_context(self, text):
        '''Extracts context from text using chatgpt api call'''
        
        SYSTEM = "You are a sophisticated AI model capable of understanding and interpreting English text. Your task is to identify and describe the overall context or main topic of the following text."
        PROMPT = f"Please identify the context of this text: \"{text}\""
        context = chatbot_completition(SYSTEM, PROMPT)
        
        return context

    def extract_key_words(self, text):
        '''Extracts key words from text using chatgpt api call'''
        
        if self.source_language == "English":
            SYSTEM = """You are a sophisticated AI model trained to identify key words and phrases in English text. Your task is to extract the most important words or phrases that capture the main points of the following text.
            Example:
            Text:"It's a beautiful sunny day at the beach."
            key words: "beautiful (όμορφος), sunny (ηλιόλουστος), day (μέρα), beach (παραλία)"
            """
        else:
            SYSTEM = """You are a sophisticated AI model trained to identify key words and phrases in Greek text. Your task is to extract the most important words or phrases that capture the main points of the following text.
            Example:
            Text:"Είναι μια όμορφη ηλιόλουστη μέρα στην παραλία."
            Key words: "όμορφη (beautiful), ηλιόλουστη (sunny), μέρα (day), παραλία (beach)"
            """
        PROMPT = f"""Please extract the key words from this text: \"{text}\ and their possible translations in {self.target_language}.
        """
        key_words = chatbot_completition(SYSTEM, PROMPT)
        return key_words


In [ ]:
# Let's instantiate the Translation object first
translator = Translation("English", "Greek")

# Then we define the text we want to translate
text = "The quick brown fox jumps over the lazy dog."

# Translate the text
translation = translator.translate(text)

# Let's print the translation
print(translation)


In [ ]:
# Let's instantiate the Translation object first
translator = Translation("Greek", "English")

# Then we define the text we want to translate
text = """Μπανιστιρτζού με έκαμε
Η αλήτισσά σου η φύση
Μπανιστιρτζού με το στανιό
Και τύφλα στο μεθύσι
Τι καλημέρα να σου πω
Άμα δε σ’ αντικρίζω
Άμα δε σ’ έχω αγκαλιά
Και να σε καβλαντίζω

Οι πρωταθλητές, στη ρακή
Δώσ’ του δώσ’ του τουφεκιές έχω τρελαθεί
Ωφ αμάν, βάι βάι
Με πήραν απ’ τον ώμο και τους είπα bye bye

Είναι πρωί και πίνω κούπες
Δε με γεμίζει ο καπνός ούτε και οι ρούφες
Είμαι δικός σου κι όλα τ’ άλλα είναι μούφες
Μούφες. Τ’ ακούς; Το λέω και με ντουντούκες
Ξυπνάω και δίνω κουτουλιές
Δυο στο μαξιλάρι και θυμάμαι όσα μου ‘πες
Έλεγα σιγά χορτάσαμε στιγμές
Τώρα με το χρόνο δες, πάλι δίνω στούκες
Κι όλο τα “θα” πάλι μπαίνουνε στη μέση
Θα τα πούμε, θα μιλήσουμε στις έξι
Θα βρεθούμε, θα και θα τα έχω παίξει
Μέσα στο κεφάλι μου μυαλό δεν έχει θέση
Μέσα στο μυαλό μου που το πήρες έτσι
Έγινα ταξίμι που έμεινε στη μέση
Έγινα έργο εθνικής, ημιτελής
Και τη φωλιά μου σίγουρα την έχω χέσει

Μπανιστιρτζού με έκαμες
Κι η απόσταση με πνίγει
Αλήτη μου με πέθανες
Κάπου δεν καταλήγει

Τι καλημέρα να σου πω
Άμα δε σ’ αντικρίζω
Άμα δε σ’ έχω αγκαλιά
Και να σε καβλαντίζω

Οι πρωταθλητές, στη ρακή
Δώσ’ του δώσ’του τουφεκιές έχω τρελαθεί
Ωφ αμάν, βάι βάι
Με πήραν απ’ τον ώμο και τους είπα bye bye

Για να χωρίσουμε εμείς
Πρέπει τη φύση σου πρώτα ν’ απαρνηθείς
Και μη μου πεις πως όλα έχουν χαλάσει
Γιατί εγώ τα άφησα όλα εντάξει
Τι φταίω, να τρέχω πάνω κάτω, α;
Και τα μούτρα σου να ρίχνεις ως το πάτωμα
Τι φάση όμως; Πες μου γιατί
Κι εγώ όπως όλοι οι άλλοι ζούμε μόνο μια ζωή
Και μη με λες καταφερτζή ούτε αλανιάρη
Και μη μου φέρεσαι λες και μου κάνεις χάρη
Δεν κάνουν λάθη μοναχά όσοι δε γεννιούνται
Αλλά και όσοι μπορούνε και θυμούνται
Μη μ’ αποπαίρνεις και ξεκόλλα καλέ
Η ζωή είναι για να ζούμε όσα οι άλλοι απαρνιούνται
Τα λόγια είναι λόγια μωρέ
Μα περισσότερα τα μάτια θα σου πούνε

Μπανιστιρτζού μ’ έχεις εκεί
Μπερμπάντισσα γυναίκα
Να πίνω ούζο και ρακή
τέσσερις πάρα δέκα

Τι καλημέρα να σου πω
Άμα δε σ’ αντικρίζω
Άμα δε σ’ έχω αγκαλιά
Και να σε καβλαντίζω

Οι πρωταθλητές στην τσικουδιά
Δώσ’ του δώσ’ του τουφεκιές πάμε ωρέ παιδιά
Ωφ αμάν, άπαπα
Με πήραν απ’ τον ώμο και τους είπα άντε γεια"""

# Translate the text
translation = translator.translate(text)

# Let's print the translation
print(translation)


In [25]:
# import logging

# # Create a logger
# logging.basicConfig(filename='example.log',level=logging.INFO)
# logger = logging.getLogger(__name__)

# # Translation chain
# class Translation:
#     def __init__(self, source_language, target_language):
#         self.source_language = source_language
#         self.target_language = target_language
        
#     def translate(self, text):
#         logger.info("Starting translation...")
        
#         logger.info("Extracting context...")
#         context = self.extract_context(text)
#         logger.info(f"Context extracted: {context}")
        
#         logger.info("Extracting key words...")
#         key_words = self.extract_key_words(text)
#         logger.info(f"Key words extracted: {key_words}")
        
#         SYSTEM = f"""You are an expert {self.target_language} translator. Your role is to carefully examine the given text, which may be written in {self.source_language}, and translate it accurately into {self.target_language}. Your translation work should account for not only literal translation but also cultural and contextual nuances. You'll be given text that has to be translated into {self.target_language}, the context of the text and key words with possible translations. Your task is to ensure the translated text preserves the original message while being grammatically correct and contextually appropriate in {self.target_language}. If the text is not in {self.source_language} you will say "Please enter text in {self.source_language} to translate" Respond using markdown."""
        
#         PROMPT = f"""
#                     Translate from {self.source_language} to {self.target_language}:
                    
#                     Example:
#                     "Given this text to translate: "Είναι μια όμορφη ηλιόλουστη μέρα στην παραλία."
#                     Translate it from Greek to English:
#                     Given the context: "This sentence is describing a pleasant weather condition at a seaside location." and key words: "όμορφη (beautiful), ηλιόλουστη (sunny), μέρα (day), παραλία (beach)"
#                     The translation is: "It's a beautiful sunny day at the beach." "
                    
#                     Now, given this text to translate: "{text}"
       
#                     Given the context: "{context}" and key words: "{key_words}"
                    
#                     The translation is:
#                     """
#         logger.info("Translating text...")
#         translation = chatbot_completition(SYSTEM, PROMPT)
#         logger.info(f"Translation completed: {translation}")
        
#         return translation, key_words
        
    
#     def extract_context(self, text):
#         '''Extracts context from text using chatgpt api call'''
#         SYSTEM = "You are a sophisticated AI model capable of understanding and interpreting Greek text. Your task is to identify and describe the overall context or main topic of the following text."
#         PROMPT = f"Please identify the context of this text: \"{text}\""
#         context = chatbot_completition(SYSTEM, PROMPT)
#         return context

#     def extract_key_words(self, text):
#         '''Extracts key words from text using chatgpt api call'''
#         SYSTEM = """You are a sophisticated AI model trained to identify key words and phrases in Greek text. Your task is to extract the most important words or phrases that capture the main points of the following text.
#         Example:
#         Text:"Είναι μια όμορφη ηλιόλουστη μέρα στην παραλία."
#         Key words: "όμορφη (beautiful), ηλιόλουστη (sunny), μέρα (day), παραλία (beach)"
#         """
#         PROMPT = f"""Please extract the key words from this text: \"{text}\ and their possible translations in {self.target_language}.
#         """
#         key_words = chatbot_completition(SYSTEM, PROMPT)
#         return key_words


In [8]:
class Transcription:
    def __init__(self, source_language, target_language):
        self.source_language = source_language
        self.target_language = target_language
        self.model = whisper.load_model("large")
        self.translate = Translation(source_language, target_language)

    def transcribe(self, audio_file):
        '''Transcribes the audio file and proofreads the transcription'''

        ### Size (in bytes) = (Sample Rate) * (Bit Depth/8) * (Number of Channels) * (Duration in seconds)
        ### Size = 44,100 * (16/8) * 1 * 60 = 5,292,000 bytes, which is approximately 5 MB.

        # Calculate the size of the audio file
        audio_file_size = os.path.getsize(audio_file)
        audio_length = audio_file_size / ( 44100 * (16/8) * 1 )
        logger.info(f"Audio file size: {audio_file_size} bytes and {audio_length} seconds")

        # Transcribe the audio
        logger.info("Starting transcription...")
        transcription = self.transcribe_audio(audio_file)
        logger.info(f"Transcription completed: {transcription}")

        # Proofread the transcription
        logger.info("Proofreading transcription...")
        proofread_transcription = self.proofread_transcription(transcription)
        logger.info(f"Proofread transcription: {proofread_transcription}")

        # Translate the transcription
        translation, _ = self.translate.translate(proofread_transcription)
        
        return proofread_transcription, translation

    def transcribe_audio(self, audio_path):
        '''Transcribes an audio file using the whisper model'''
        
        # Check if file size is less than 25 MB
        file_size = os.path.getsize(audio_path) / (1024 * 1024) # size in MB
        if file_size > 25:
            raise ValueError(f"The file size is {file_size:.2f} MB which exceeds the 25 MB limit.")

        # Check if file extension is supported
        supported_file_types = ['mp3', 'mp4', 'mpeg', 'mpga', 'm4a', 'wav', 'webm']
        file_extension = os.path.splitext(audio_path)[1][1:]
        if file_extension not in supported_file_types:
            raise ValueError(f"The file type '{file_extension}' is not supported. Please use one of the following: {', '.join(supported_file_types)}.")

        # If checks pass, transcribe the audio
        audio_file = open(audio_path, "rb")
        transcript = openai.Audio.transcribe("whisper-1", audio_file)

        # Always close the file you opened
        audio_file.close()

        return transcript
        
        # result = self.model.transcribe(audio_file)        
        # return result["text"]

    def proofread_transcription(self, transcription):
        '''Improves the transcription quality using GPT-4'''
        
        SYSTEM = """You are an expert {self.source_langauge} transcription agent. In particular, an advanced language model trained to proofread and correct text based on grammar, syntax, and coherence in context. Your role is to carefully examine the given text in {self.source_langauge}, identify semantic errors resulting from transcription, and then correct these errors. Use your knowledge of {self.source_language} language and syntax to ensure the accuracy and coherence of the transcriptions. Your task is crucial for the quality and usefulness of the transcriptions. You will be given {self.source_langauge} text that has been transcribed but may contain errors. Your job is to fix these errors to the best of your ability. Respond using markdown.
        
        You will work through proofreading and correcting the transcription in the following way step by step:
        
        Identify the context: [context]
        Identify key words: [key words]
        Correct the transcription: [transcription]
        
        You will not add any additional detail to the transcription. You will only correct the transcription. You will only return the transcription. You will not return the context or key words.
        
        Write the transcription here:
               
        
        """
        
        PROMPT = f"Please proofread and correct this transcription: \"{transcription}\""
        proofread_transcription = chatbot_completition(SYSTEM, PROMPT)
        return proofread_transcription

In [9]:
from pydub import AudioSegment

transcriber = Transcription("Greek", "English")

audio_path = "greek1.mp3"

# audio = AudioSegment.from_mp3(audio_path)

proofread, translation = transcriber.transcribe(audio_path)
print(proofread)
print(translation)

"Γεια σου γιαγιά! Τι κάνεις? Καλώς το παλικάρι μου! Μόλις τώρα έφτιαξα το αγαπημένο σου φαγητό, τα κουπέπια! Τι τυχερός που είμαι! Πώς τα κάνεις γιαγιά? Πρώτα απ' όλα, χρειάζεσαι αμπελόφυλλα. Όταν ήμουν μικρή, η μητέρα μου μάζευε τα αμπελόφυλλα από την κλιματαριά μας. Τότε όλα τα σπίτια είχαν κλιματαριές, ενώ τώρα αγοράζουν τα φύλλα από την αγορά. Τα περισσότερα υλικά που χρησιμοποιούσαμε παλιά στο μαγείρεμα

"Hello grandmother! How are you? Welcome, my boy! I've just made your favorite meal, koupepia! How lucky I am! How do you make it, grandmother? First of all, you need vine leaves. When I was young, my mother used to gather vine leaves from our vineyard. Back then, every house had a vineyard, while now people buy the leaves from the market. Most of the ingredients we used to use in cooking, we got from our own home garden."


In [41]:
# gradio test code
# structure of the app
# translate tab translalte and vocab check to select language

source_language = "English"  # Default language
target_language = "Greek" if source_language == "English" else "English"
translator = Translation(source_language, target_language)

def update_language(widget, state):
    global source_language, target_language, translator
    source_language = widget
    target_language = "Greek" if source_language == "English" else "English"
    translator = Translation(source_language, target_language)
    return widget

def main():
       
    # Translate tab
    with gr.Blocks() as chatbot_demo:
        # Create a state to hold some global variable    
        source_language_state = gr.State(False)
        source_language_widget = gr.Radio(["English", "Greek"], label="Source Language",value="English")
        source_language_widget.change(update_language, inputs=[source_language_widget, source_language_state], outputs=[source_language_state])
        
 
        with gr.Row():  # Define a Row
            source_language_widget  # Add your widget within the Row

        with gr.Row():  # Define another Row
            with gr.Column():
                chatbot = gr.Chatbot([], elem_id="Translation bot", label = 'Translation bot').style(height=250)
                msg = gr.Textbox()
                clear = gr.Button("Clear")

                def respond(message, chat_history):

                    bot_message, key_words = translator.translate(message)  # Use your chatbot_completition function here
                    chat_history.append((message, bot_message))
                    return "", chat_history, key_words

                msg.submit(respond, [msg, chatbot], [msg, chatbot])
                clear.click(lambda: None, None, chatbot, queue=False)
        
                with gr.Accordion("Greek examples", open=False):
                    gr.Examples(
                        examples=["Είναι μια όμορφη ηλιόλουστη μέρα στην παραλία.",
                                    "Αυτός είναι ο καλύτερος φίλος μου",
                                    "Το παιδί έχει πολλά παιχνίδια"],
                        inputs=msg
                    )
            
            with gr.Column():
                key_words_textbox = gr.Textbox(label='Vocab',interactive= False)  # Define a textbox to show the keywords
                msg.submit(respond, [msg, chatbot], [msg, chatbot, key_words_textbox]) 
    # # Transcription tab
    # # set language pair with checkboxes
    # source_language = gr.inputs.CheckboxGroup(["English", "Greek"], label="Source Language")
    # target_language = "Greek" if source_language == "English" else "English"
    

    # # create audio input for microphone 
    # audio_input = gr.inputs.Audio(source="microphone", type="filepath")
    
    # output_text = gr.outputs.Textbox()
    
    # transcription_interface = gr.Interface(
    #     fn= transcribe_audio,
    #     inputs=audio_input,
    #     outputs=output_text,
    #     title="Audio Transcription",
    #     description="Upload an audio file or record audio using the microphone, and hit the 'Submit' button.",
    #     live=True
    # )
    
    chatbot_demo.launch(share=True)

if __name__ == "__main__":
    main()
    
    
    
    

c:\Python311\Lib\site-packages\gradio\components\chatbot.py:228: UserWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7882
Running on public URL: https://e0f14a92a2ad72506b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [4]:
def transcribe_audio(audio_file):
    model = whisper.load_model("small")
    result = model.transcribe(audio_file)
    return result["text"]

def chatbot_completition(user_content):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert Greek translator and transcription agent. Your role is to carefully examine the given text in Greek, identify semantic errors resulting from transcription, and then correct these errors. Use your knowledge of Greek language and syntax to ensure the accuracy and coherence of the transcriptions. Your task is crucial for the quality and usefulness of the transcriptions. You will be given Greek text that has been transcribed but may contain errors. Your job is to fix these errors to the best of your ability. Respond using markdown."},
            {"role": "user", "content": user_content},
        ],
    )
    return response["choices"][0]["message"]["content"]

def chatbot_english_to_greek(user_content):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert Greek translator. Your role is to carefully examine the given text, which may be written in English, and translate it accurately into Greek. Your translation work should account for not only literal translation but also cultural and contextual nuances. You'll be given text that has been translated into Greek, but it may contain errors due to literal or out-of-context translations. Your task is to correct these errors, ensuring the translated text preserves the original message while being grammatically correct and contextually appropriate in Greek.  Respond using markdown."},
            {"role": "user", "content": user_content},
        ],
    )
    return response["choices"][0]["message"]["content"]

# def chatbot_greek_to_english(user_content):
#     response = openai.ChatCompletion.create(
#         model="gpt-4",
#         messages=[
#             {"role": "system", "content": "You are an expert Greek translator. Your role is to carefully examine the given text, which may be written in English, and translate it accurately into Greek. Your translation work should account for not only literal translation but also cultural and contextual nuances. You'll be given text that has been translated into Greek, but it may contain errors due to literal or out-of-context translations. Your task is to correct these errors, ensuring the translated text preserves the original message while being grammatically correct and contextually appropriate in Greek.  Respond using markdown."},
#             {"role": "user", "content": user_content},
#         ],
#     )
#     return response["choices"][0]["message"]["content"]


def handle_audio_inputs(audio_upload, audio_microphone):
    if audio_upload is not None:
        return audio_upload
    else:
        return audio_microphone


In [4]:
def main():
    audio_input = gr.inputs.Audio(source="upload", type="filepath")
    audio_input_microphone = gr.inputs.Audio(source="microphone", type="filepath")

    output_text = gr.outputs.Textbox()

    transcription_interface = gr.Interface(
        fn=lambda audio_upload, audio_microphone: transcribe_audio(handle_audio_inputs(audio_upload, audio_microphone)),
        inputs=[audio_input, audio_input_microphone],
        outputs=output_text,
        title="Audio Transcription",
        description="Upload an audio file or record audio using the microphone, and hit the 'Submit' button.",
        live=True
    )

    with gr.Blocks() as chatbot_demo:
        chatbot = gr.Chatbot([], elem_id="chatbot").style(height=250)
        msg = gr.Textbox()
        clear = gr.Button("Clear")

        def respond(message, chat_history):
            bot_message = chatbot_completition(message)  # Use your chatbot_completition function here
            chat_history.append((message, bot_message))
            return "", chat_history

        msg.submit(respond, [msg, chatbot], [msg, chatbot])
        clear.click(lambda: None, None, chatbot, queue=False)

    with gr.Blocks() as chatbot_demo2:
        chatbot2 = gr.Chatbot([], elem_id="chatbot2").style(height=250)
        msg2 = gr.Textbox()
        clear2 = gr.Button("Clear")

        def respond2(message, chat_history):
            bot_message2 = chatbot_completition_2(message)  # Use your chatbot_completition function here
            chat_history.append((message, bot_message2))
            return "", chat_history

        msg2.submit(respond2, [msg2, chatbot2], [msg2, chatbot2])
        clear2.click(lambda: None, None, chatbot2, queue=False)

    tabbed_interface = gr.TabbedInterface([transcription_interface, chatbot_demo, chatbot_demo2], ["Transcription", "Chatbot transcription", "Chatbot translate"])

    tabbed_interface.launch(share=True)

if __name__ == '__main__':
    main()

c:\Python311\Lib\site-packages\gradio\inputs.py:319: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
c:\Python311\Lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
c:\Python311\Lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
c:\Python311\Lib\site-packages\gradio\blocks.py:255: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://243ad3db81b4c62de9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
